# Radar Frame Labeling Tool
Samples events from the TorNet catalog and lets you label each frame as:
- `tornado_hook`
- `tornado_no_hook`
- `non_tornadic`

Results are saved to a CSV after every label.

In [9]:
import os
import random
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path
import boto3

## Configuration

In [24]:
s3_client = boto3.client('s3')
bucket_name = 'ml-cloud-project-data'

In [23]:
# ── Change these to match your setup ──────────────────────────────────────────
DATA_DIR    = "s3://ml-cloud-project-data"          # root dir containing tornet_2013/, tornet_2014/, etc.
CATALOG_CSV = "s3://ml-cloud-project-data/tornet_2013/catalog.csv"
SPLIT            = "train"               # 'train' or 'test'
N_PER_CATEGORY   = 50                    # unique events (by event_id) per category
FRAMES_PER_EVENT = None                  # frames to subsample per event (None = all)
OUTPUT_CSV       = "labels.csv"
SWEEP_IDX        = 0                     # which sweep to display (0 = lowest elevation)
SEED             = 42
# ──────────────────────────────────────────────────────────────────────────────

VARIABLES_TO_PLOT = ['DBZ', 'VEL']

COLORMAP_MAP = {
    'DBZ': ('gist_ncar', -10, 75),
    'VEL': ('bwr',       -30, 30),
}

LABEL_OPTIONS = ['tornado_hook', 'tornado_no_hook', 'non_tornadic']

## Event Sampling from Catalog

In [32]:
def sample_events_from_catalog(catalog_csv, data_dir, split, n_per_category, seed=42):
    """
    Sample n_per_category TOR events and n_per_category non-tornadic events.
    Non-tornadic events are drawn equally from NUL and WRN (both required).
    Only the 'train' split is used.
    """
    random.seed(seed)
    catalog = pd.read_csv(catalog_csv)
    catalog = catalog[catalog['type'] == split].copy()

    def sample_category(cat_label, n):
        rows       = catalog[catalog['category'] == cat_label]
        unique_ids = rows['event_id'].unique().tolist()
        sampled    = random.sample(unique_ids, min(n, len(unique_ids)))
        files = (
            rows[rows['event_id'].isin(sampled)]
            .groupby('event_id')['filename']
            .first()
            .tolist()
        )
        print(f"  {cat_label}: {len(files)} events sampled from {len(unique_ids)} available")
        return files

    print(f"Sampling from split='{split}' ...")
    tor_files = sample_category('TOR', n_per_category)

    # Non-tornadic: half from NUL, half from WRN — both required
    half = n_per_category // 2
    nul_files = sample_category('NUL', half)
    wrn_files = sample_category('WRN', half)

    def resolve_path(rel_filename):
        parts = Path(rel_filename).parts   # ('train', '2013', 'filename.nc')
        year  = parts[1] if len(parts) >= 3 else ''
        return f"{data_dir}/tornet_{year}/{rel_filename}"

    all_files = [resolve_path(f) for f in tor_files + nul_files + wrn_files]
    random.shuffle(all_files)
    return all_files


def load_event_frames(filepath, frames_per_event=None, seed=42):
    """Open a .nc file and return (dataset, list_of_frame_indices)."""
    ds = xr.open_dataset(filepath, engine='h5netcdf')
    n_frames = ds.sizes.get('time', 1)
    indices  = list(range(n_frames))
    if frames_per_event and len(indices) > frames_per_event:
        random.seed(seed)
        indices = sorted(random.sample(indices, frames_per_event))
    return ds, indices


all_event_files = sample_events_from_catalog(
    CATALOG_CSV, DATA_DIR, SPLIT, N_PER_CATEGORY, SEED
)
print(f"\nTotal events queued : {len(all_event_files)}")
print(f"Example path        : {all_event_files[0] if all_event_files else 'n/a'}")

Sampling from split='train' ...
  TOR: 50 events sampled from 6655 available
  NUL: 25 events sampled from 13620 available
  WRN: 25 events sampled from 3514 available

Total events queued : 100
Example path        : s3://ml-cloud-project-data/tornet_2014/train/2014/TOR_140404_020014_KPAH_504204_P0.nc


## Labeling UI

In [31]:
def plot_frame(ds, frame_idx, event_name, frame_num, total_frames):
    """Render DBZ and VEL for one frame in polar (range-azimuth) coordinates."""
    vars_present = [v for v in VARIABLES_TO_PLOT if v in ds]

    fig, axes = plt.subplots(
        1, len(vars_present),
        figsize=(10, 4.5),
        facecolor='#0d0d0d',
        subplot_kw=dict(projection='polar'),
    )
    if len(vars_present) == 1:
        axes = [axes]

    fig.suptitle(
        f"{event_name}  |  Frame {frame_num}/{total_frames}",
        color='white', fontsize=12, fontweight='bold',
    )

    # Build polar coordinate grids once (same for all variables in this ds)
    az_deg = ds['azimuth'].values          # (azimuth,)  degrees
    r_km   = ds['range'].values / 1000.0   # (range,)    km
    AZ, R  = np.meshgrid(np.deg2rad(az_deg), r_km)   # both (range, azimuth)

    for ax, var in zip(axes, vars_present):
        ax.set_facecolor('#111')

        data = ds[var].values   # (time, azimuth, range, sweep)
        if data.ndim == 4:
            frame_data = data[frame_idx, :, :, SWEEP_IDX]   # (azimuth, range)
        elif data.ndim == 3:
            frame_data = data[frame_idx, :, :]               # (azimuth, range)
        else:
            frame_data = data

        # pcolormesh expects (range, azimuth) — transpose
        cmap, vmin, vmax = COLORMAP_MAP.get(var, ('viridis', None, None))
        mesh = ax.pcolormesh(
            AZ, R, frame_data.T,
            cmap=cmap, vmin=vmin, vmax=vmax,
            shading='auto',
        )

        # North up, clockwise (standard radar convention)
        ax.set_theta_zero_location('N')
        ax.set_theta_direction(-1)
        ax.set_ylim(0, r_km.max())

        # Restrict angular extent to the actual sweep
        ax.set_thetamin(az_deg.min())
        ax.set_thetamax(az_deg.max())

        # Styling
        ax.tick_params(colors='white')
        ax.yaxis.label.set_color('white')
        ax.title.set_color('white')
        ax.set_title(var, color='white', fontsize=11, pad=12)

        cb = plt.colorbar(mesh, ax=ax, fraction=0.046, pad=0.08)
        cb.ax.tick_params(colors='white')
        cb.set_label(
            'dBZ' if var == 'DBZ' else 'm/s',
            color='white', rotation=270, labelpad=12,
        )

    plt.tight_layout()
    return fig



In [34]:
class NotebookLabeler:
    """Widget-based frame labeling tool. Call .start() to begin."""

    def __init__(self, event_files, output_csv=OUTPUT_CSV, frames_per_event=FRAMES_PER_EVENT):
        self.event_files      = event_files
        self.output_csv       = output_csv
        self.frames_per_event = frames_per_event
        self.results          = []
        self._queue           = []
        self._pos             = 0
        self._ds_cache        = {}
        self._hook_count      = 0   # ← running tornado-with-hook counter

        # ── Widgets ──────────────────────────────────────────────
        self.btn_hook   = widgets.Button(description='🌀 Tornado w/ Hook',
                                         layout=widgets.Layout(width='200px', height='44px'),
                                         style=dict(button_color='#1a3a6b', font_weight='bold'))
        self.btn_nohook = widgets.Button(description='⚡ Tornado No Hook',
                                         layout=widgets.Layout(width='200px', height='44px'),
                                         style=dict(button_color='#1a5a6b', font_weight='bold'))
        self.btn_non    = widgets.Button(description='☁️ Non-Tornadic',
                                         layout=widgets.Layout(width='200px', height='44px'),
                                         style=dict(button_color='#2d4a1e', font_weight='bold'))
        self.btn_skip   = widgets.Button(description='⏭ Skip Event',
                                         layout=widgets.Layout(width='130px', height='44px'),
                                         style=dict(button_color='#4a3800', font_weight='bold'))
        self.btn_undo   = widgets.Button(description='↩ Undo',
                                         layout=widgets.Layout(width='100px', height='44px'),
                                         style=dict(button_color='#3a1e3a', font_weight='bold'))

        self.progress_bar   = widgets.IntProgress(min=0, max=1, value=0,
                                                   layout=widgets.Layout(width='90%'),
                                                   style={'bar_color': '#2a72c8'})
        self.progress_label = widgets.Label(value='0/0 frames')
        self.hook_counter = widgets.Label(
            value='🌀 Hooks labeled: 0',
            style={'font_weight': 'bold'},
            layout=widgets.Layout(margin='0 0 0 16px', min_width='180px'),  # ← add min_width
        )
        self.status_label   = widgets.Label()
        self.plot_output    = widgets.Output()

        self.btn_hook.on_click(lambda b:   self._label('tornado_hook'))
        self.btn_nohook.on_click(lambda b: self._label('tornado_no_hook'))
        self.btn_non.on_click(lambda b:    self._label('non_tornadic'))
        self.btn_skip.on_click(lambda b:   self._skip_event())
        self.btn_undo.on_click(lambda b:   self._undo())

        self._ui = widgets.VBox([
            widgets.HBox([self.progress_bar, self.progress_label, self.hook_counter]),
            self.plot_output,
            widgets.HBox(
                [self.btn_hook, self.btn_nohook, self.btn_non, self.btn_skip, self.btn_undo],
                layout=widgets.Layout(justify_content='center', gap='8px', margin='8px 0'),
            ),
            self.status_label,
        ])

    def start(self):
        self._build_queue()
        if not self._queue:
            print("Queue is empty — check that DATA_DIR and CATALOG_CSV are correct.")
            return
        self.progress_bar.max = len(self._queue)
        display(self._ui)
        self._show_current()

    @property
    def labels_df(self):
        return pd.DataFrame(self.results)

    # ── Internal ──────────────────────────────────────────────────

    def _build_queue(self):
        for fp in self.event_files:
            try:
                ds, indices = load_event_frames(fp, self.frames_per_event, SEED)
                self._ds_cache[fp] = (ds, indices)
                for fi in indices:
                    self._queue.append((fp, fi))
            except Exception as e:
                print(f"Could not load {Path(fp).name}: {e}")
        print(f"Total frames to label: {len(self._queue)}")

    def _show_current(self):
        if self._pos >= len(self._queue):
            self._finish()
            return
        fp, frame_idx = self._queue[self._pos]
        ds, indices   = self._ds_cache[fp]
        event_name    = Path(fp).stem
        frame_num     = indices.index(frame_idx) + 1
        total_frames  = len(indices)

        self.progress_bar.value   = self._pos
        self.progress_label.value = f"  {self._pos}/{len(self._queue)} frames"

        with self.plot_output:
            clear_output(wait=True)
            fig = plot_frame(ds, frame_idx, event_name, frame_num, total_frames)
            plt.show()

    def _label(self, label):
        if self._pos >= len(self._queue):
            return
        fp, frame_idx = self._queue[self._pos]
        self.results.append({
            'filepath':  fp,
            'event_id':  Path(fp).stem,
            'frame_idx': frame_idx,
            'label':     label,
        })

        # Update hook counter
        if label == 'tornado_hook':
            self._hook_count += 1
            self.hook_counter.value = f'🌀 Hooks labeled: {self._hook_count}'

        self.status_label.value = f"✓ Labeled as: {label}"
        self._pos += 1
        self._auto_save()
        self._show_current()

    def _skip_event(self):
        if self._pos >= len(self._queue):
            return
        current_event = self._queue[self._pos][0]
        while self._pos < len(self._queue) and self._queue[self._pos][0] == current_event:
            self._pos += 1
        self.status_label.value = f"⏭ Skipped: {Path(current_event).stem}"
        self._show_current()

    def _undo(self):
        if self.results and self._pos > 0:
            last = self.results.pop()
            self._pos -= 1
            # Decrement hook counter if we're undoing a hook label
            if last['label'] == 'tornado_hook':
                self._hook_count = max(0, self._hook_count - 1)
                self.hook_counter.value = f'🌀 Hooks labeled: {self._hook_count}'
            self.status_label.value = "↩ Undid last label"
            self._show_current()

    def _auto_save(self):
        if self.results:
            pd.DataFrame(self.results).to_csv(self.output_csv, index=False)

    def _finish(self):
        self._auto_save()
        with self.plot_output:
            clear_output(wait=True)
            print("✅ All frames labeled!")
        self.progress_bar.value   = len(self._queue)
        self.progress_label.value = f"  {len(self._queue)}/{len(self._queue)} frames"
        self.status_label.value   = f"💾 Saved to {self.output_csv}"
        df = pd.DataFrame(self.results)
        print("\n── Label distribution ──")
        print(df['label'].value_counts().to_string())
        print(f"\nTotal labeled frames : {len(df)}")
        print(f"🌀 Tornado w/ Hook   : {self._hook_count}")


## Run the Labeler
Click the buttons beneath each frame to assign a label. Progress autosaves after every label.

In [ ]:
labeler = NotebookLabeler(
    event_files=all_event_files,
    output_csv=OUTPUT_CSV,
    frames_per_event=FRAMES_PER_EVENT,
)
labeler.start()

Could not load WRN_170901_203725_KRAX_1079702n_H2.nc: ml-cloud-project-data/tornet_2017/train/2017/WRN_170901_203725_KRAX_1079702n_H2.nc
Could not load NUL_170510_035621_KFDX_1078685n_H4.nc: ml-cloud-project-data/tornet_2017/train/2017/NUL_170510_035621_KFDX_1078685n_H4.nc
Could not load WRN_170929_205343_KABX_1079824n_Y5.nc: ml-cloud-project-data/tornet_2017/train/2017/WRN_170929_205343_KABX_1079824n_Y5.nc
Could not load NUL_171022_100350_KLIX_1079872n_V4.nc: ml-cloud-project-data/tornet_2017/train/2017/NUL_171022_100350_KLIX_1079872n_V4.nc
Could not load TOR_170121_163418_KFFC_673683_I2.nc: ml-cloud-project-data/tornet_2017/train/2017/TOR_170121_163418_KFFC_673683_I2.nc
Total frames to label: 380


## Review Labels

In [36]:
df = labeler.labels_df
print(df['label'].value_counts())
df

label
tornado_hook       14
tornado_no_hook    12
non_tornadic       12
Name: count, dtype: int64


,filepath,event_id,frame_idx,label
0,s3://ml-cloud-project-data/tornet_2014/train/2...,TOR_140404_020014_KPAH_504204_P0,0,tornado_hook
1,s3://ml-cloud-project-data/tornet_2014/train/2...,TOR_140404_020014_KPAH_504204_P0,1,tornado_hook
2,s3://ml-cloud-project-data/tornet_2014/train/2...,TOR_140404_020014_KPAH_504204_P0,2,tornado_hook
3,s3://ml-cloud-project-data/tornet_2014/train/2...,TOR_140404_020014_KPAH_504204_P0,3,tornado_no_hook
4,s3://ml-cloud-project-data/tornet_2016/train/2...,TOR_160523_003937_KMAF_631193_Y0,0,tornado_hook
5,s3://ml-cloud-project-data/tornet_2016/train/2...,TOR_160523_003937_KMAF_631193_Y0,1,tornado_hook
6,s3://ml-cloud-project-data/tornet_2016/train/2...,TOR_160523_003937_KMAF_631193_Y0,2,tornado_hook
7,s3://ml-cloud-project-data/tornet_2016/train/2...,TOR_160523_003937_KMAF_631193_Y0,3,tornado_hook
8,s3://ml-cloud-project-data/tornet_2019/train/2...,NUL_190630_033051_KMVX_829230s_M3,0,non_tornadic
9,s3://ml-cloud-project-data/tornet_2019/train/2...,NUL_190630_033051_KMVX_829230s_M3,1,non_tornadic
